# References

- https://www.sciencedirect.com/science/article/pii/S1110016811000421#!
- https://salman-h-khan.github.io/papers/TPAMI15.pdf
- https://link.springer.com/chapter/10.1007/978-3-319-21978-3_52
- https://ieeexplore.ieee.org/abstract/document/5196726/metrics#metrics
- https://www.sciencedirect.com/science/article/pii/S2214241X15001789

- https://ieeexplore.ieee.org/document/7009877?reload=true&arnumber=7009877

# Parameters

In [ ]:
REGION_NAMES = ['GRASS', 'SIDEWALK', 'BUILDING', 'GRAVEL']

# don't change
REFERENCE_MISSION = 2

# Imports and Functions

In [ ]:
import copy
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage import exposure
from skimage.morphology import area_opening, disk, square
import sys

In [ ]:
# sys.path.insert(0, '.py')

In [ ]:
def plot_images(image_list, title_list=[], grid='off', size_per_image=10):
    fig, axes = plt.subplots(nrows=1, ncols=len(image_list), figsize=(size_per_image*len(image_list), size_per_image*1))
    if len(image_list) == 1:
        axes = [axes]

    for i, ax in enumerate(axes):
        ax.imshow(cv2.cvtColor(image_list[i], cv2.COLOR_BGR2RGB))
        if len(title_list) > 0:
            ax.set_title(title_list[i])
        ax.axis(grid)

In [ ]:
def plot_mission_region_sample_images(mission_numbers, mission_images=[], region_images=[], sample_images=[], display_missions=True, display_regions=True, display_samples=True):
    if display_missions:
        if len(mission_images) > 0:
            print('Displaying missions')
            plot_images(mission_images, ['Mission ' + str(mission_number) for mission_number in mission_numbers])
        else:
            print('[Error] No mission images available to display')
    
    if display_regions:
        if len(region_images) > 0:
            print('Displaying regions')
            plot_images(region_images, ['Region ' + str(mission_number) for mission_number in mission_numbers])
        else:
            print('[Error] No region images available to display')
    
    if display_samples:
        if len(sample_images) > 0:
            print('Displaying samples')
            plot_images(sample_images, ['Sample ' + str(mission_number) for mission_number in mission_numbers])
        else:
            print('[Error] No sample images available to display')

In [ ]:
def get_mission_file_path(mission_number):
    return '..\\missions\\mission_' + str(mission_number) + '\\mission_' + str(mission_number) + '_'

In [ ]:
def get_global_image(mission_number, get_normalized=True):
    if get_normalized:
        mission = np.load(get_mission_file_path(mission_number) + 'normalized_image.npy')
    else:
        mission = np.load(get_mission_file_path(mission_number) + 'aligned_image.npy')
    
    return mission

In [ ]:
def get_region_and_sample_image(mission_number, region_name, get_normalized=True):
    if get_normalized:
        region = np.load(get_mission_file_path(mission_number) + 'normalized_region_image_' + region_name + '.npy')
        sample = np.load(get_mission_file_path(mission_number) + 'normalized_sample_image_' + region_name + '.npy')
    else:
        region = np.load(get_mission_file_path(mission_number) + 'aligned_region_image_' + region_name + '.npy')
        sample = np.load(get_mission_file_path(mission_number) + 'aligned_sample_image_' + region_name + '.npy')
    
    return region, sample

# Output Functions

In [ ]:
from skimage.measure import label, regionprops

def create_bounding_boxes(masks, output_images, display_bbox=True, display_output=True, num_bbox_iterations=1, bbox_color=(255, 0, 0)):

    temp = copy.deepcopy(masks)
    temp_bounding_box_masks = []

    num_bbox = []
    
    for k in range(0, num_bbox_iterations):
        print('Iter:', k)
        bounding_box_masks = []

        for i in range(0, len(temp)):
            label_ = label(temp[i])
            props_ = regionprops(label_)
            bounding_mask = np.zeros((temp[i].shape)).astype(np.uint8)

            print('# bbox:', len(props_))
            if k == num_bbox_iterations-1:
                num_bbox.append(len(props_))

            factor = 0
            for prop in props_:
        #         print('bbox', prop.bbox)
                if k == num_bbox_iterations-1:
                    cv2.rectangle(bounding_mask, (prop.bbox[1], prop.bbox[0]), (prop.bbox[3], prop.bbox[2]), (255, 0, 0), 2)
                else:
                    cv2.rectangle(bounding_mask, (prop.bbox[1]-factor, prop.bbox[0]-factor), (prop.bbox[3]+2*factor, prop.bbox[2]+2*factor), bbox_color, 2)

            bounding_box_masks.append(bounding_mask)

        temp_bounding_box_masks.append(copy.deepcopy(bounding_box_masks))
        temp = copy.deepcopy(bounding_box_masks)
    
    if display_bbox:
        for k in range(0, num_bbox_iterations):
            plot_images(temp_bounding_box_masks[k], ['bounding_box_masks ' + str(mission_number) for mission_number in mission_numbers])
    
    results = []
    for i in range(0, len(mission_numbers)):
        result = copy.deepcopy(output_images[i])
        result[temp_bounding_box_masks[num_bbox_iterations-1][i].astype(bool), :] = [0, 0, 255]
        results.append(result)
        
    if display_output:
        # differences wrt mission 2
        for i in range(0, len(mission_numbers)):
            plot_images([histogram_mission_images_bgr[0], results[i]], ['reference', 'result ' + str(mission_numbers[i])], size_per_image=3)
            
    return bounding_box_masks, num_bbox, results

In [ ]:
import matplotlib as mpl
from PIL import ImageColor

def color_fader(cs, mix=0):
    if len(cs) == 2:
        c1 = np.array(mpl.colors.to_rgb(cs[0]))
        c2 = np.array(mpl.colors.to_rgb(cs[1]))
        return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
    elif len(cs) == 3:
        c1 = np.array(mpl.colors.to_rgb(cs[0]))
        c2 = np.array(mpl.colors.to_rgb(cs[1]))
        c3 = np.array(mpl.colors.to_rgb(cs[2]))
        if mix <= .5:
            mix *= 2 # create 0-1 range
            return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
        else:
            mix -= .5 # create 0-1 range
            mix *= 2
            return mpl.colors.to_hex((1-mix)*c2 + mix*c3)

def create_color_buckets(colors=['#FF0000', '#00FF00'], num_buckets=10, display=True): #RGB
    color_buckets = []

    if display:
        fig, ax = plt.subplots(figsize=(4, 2))
    
    for x in range(0, num_buckets):
        X = color_fader(colors, x/(num_buckets-1))
        
        if display:
            ax.axvline(x, color=X, linewidth=4)
        
        color_buckets.append(ImageColor.getcolor(X, "RGB")[::-1]) # convert RGB to BGR

    if display:
        plt.axis('off')
        plt.show()

    return color_buckets

# Start

In [ ]:
mission_numbers = [REFERENCE_MISSION, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]
region_type = 'BUILDING'
get_normalized = False

In [ ]:
mission_images_bgr = []
region_images_bgr = []
# sample_images = []

for mission_number in mission_numbers:
    if not os.path.isdir('..\\missions\\mission_' + str(mission_number)):
        print('[Error] Mission %d directory has not been created' % (mission_number))
        break
    
    mission_image = get_global_image(mission_number, get_normalized)
    mission_images_bgr.append(mission_image)
    
    region_image, sample_image = get_region_and_sample_image(mission_number, region_type, get_normalized)
    region_images_bgr.append(region_image)
#     sample_images.append(sample_image)
    
# all lengths and dimensions should match
print(mission_images_bgr[0].shape)
print(region_images_bgr[0].shape)
# print(sample_images[0].shape)
print(len(mission_images_bgr))
print(len(region_images_bgr))
# print(len(sample_images))

# Missions, Regions, Samples in BGR

In [ ]:
plot_images(mission_images_bgr, ['mission_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(region_images_bgr, ['region_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(sample_images, ['sample_images ' + str(mission_number) for mission_number in mission_numbers])

# Histogram Matching

In [ ]:
# https://automaticaddison.com/how-to-do-histogram-matching-using-opencv/

def calculate_cdf(histogram):
    """
    This method calculates the cumulative distribution function
    :param array histogram: The values of the histogram
    :return: normalized_cdf: The normalized cumulative distribution function
    :rtype: array
    """
    # Get the cumulative sum of the elements
    cdf = histogram.cumsum()
 
    # Normalize the cdf
    normalized_cdf = cdf / float(cdf.max())
 
    return normalized_cdf
 
def calculate_lookup(new_cdf, ref_cdf):
    """
    This method creates the lookup table
    :param array new_cdf: The cdf for the source image
    :param array ref_cdf: The cdf for the reference image
    :return: lookup_table: The lookup table
    :rtype: array
    """
    lookup_table = np.zeros(256)
    lookup_val = 0
    for new_pixel_val in range(len(new_cdf)):
        lookup_val
        for ref_pixel_val in range(len(ref_cdf)):
            if ref_cdf[ref_pixel_val] >= new_cdf[new_pixel_val]:
                lookup_val = ref_pixel_val
                break
        lookup_table[new_pixel_val] = lookup_val
    return lookup_table

def match_histograms(ref_image, new_image):
    """
    This method matches the new image histogram to the
    reference signal
    :param image new_image: The original source image
    :param image  ref_image: The reference image
    :return: image_after_matching
    :rtype: image (array)
    """
    # Split the images into the different color channels
    # b means blue, g means green and r means red
    new_b, new_g, new_r = cv2.split(new_image)
    ref_b, ref_g, ref_r = cv2.split(ref_image)
 
    # Compute the b, g, and r histograms separately
    # The flatten() Numpy method returns a copy of the array c
    # collapsed into one dimension.
    new_hist_blue, bin_0 = np.histogram(new_b.flatten(), 256, [0,256])
    new_hist_green, bin_1 = np.histogram(new_g.flatten(), 256, [0,256])
    new_hist_red, bin_2 = np.histogram(new_r.flatten(), 256, [0,256])    
    ref_hist_blue, bin_3 = np.histogram(ref_b.flatten(), 256, [0,256])    
    ref_hist_green, bin_4 = np.histogram(ref_g.flatten(), 256, [0,256])
    ref_hist_red, bin_5 = np.histogram(ref_r.flatten(), 256, [0,256])
 
    # Compute the normalized cdf for the source and reference image
    new_cdf_blue = calculate_cdf(new_hist_blue)
    new_cdf_green = calculate_cdf(new_hist_green)
    new_cdf_red = calculate_cdf(new_hist_red)
    ref_cdf_blue = calculate_cdf(ref_hist_blue)
    ref_cdf_green = calculate_cdf(ref_hist_green)
    ref_cdf_red = calculate_cdf(ref_hist_red)
 
    # Make a separate lookup table for each color
    blue_lookup_table = calculate_lookup(new_cdf_blue, ref_cdf_blue)
    green_lookup_table = calculate_lookup(new_cdf_green, ref_cdf_green)
    red_lookup_table = calculate_lookup(new_cdf_red, ref_cdf_red)
 
    # Use the lookup function to transform the colors of the original
    # source image
    blue_after_transform = cv2.LUT(new_b, blue_lookup_table)
    green_after_transform = cv2.LUT(new_g, green_lookup_table)
    red_after_transform = cv2.LUT(new_r, red_lookup_table)
 
    # Put the image back together
    image_after_matching = cv2.merge([blue_after_transform, green_after_transform, red_after_transform])
    image_after_matching = cv2.convertScaleAbs(image_after_matching)
 
    return image_after_matching

In [ ]:
histogram_mission_images_bgr = []
histogram_region_images_bgr = []

for i in range(0, len(mission_numbers)):
    histogram_mission_images_bgr.append(match_histograms(mission_images_bgr[0], mission_images_bgr[i]))
    histogram_region_images_bgr.append(match_histograms(region_images_bgr[0], region_images_bgr[i]))

# Intensity Matching

In [ ]:
# intensity_mission_images = []

# for i in range(0, len(mission_numbers)):
#     p1, p2 = np.percentile(mission_images[i], (5, 95))
#     intensity_mission_images.append(exposure.rescale_intensity(mission_images[i], in_range=(p1, p2)))

# Average Mission

In [ ]:
# # take the average
# def averager(images):
#     A = np.sum(images, axis=0)
#     A = A / len(images)
#     A = A.astype(np.uint8)
#     return A

In [ ]:
# indices = [mission_numbers.index(n) for n in [2, 6, 8, 9]]
# images = [histogram_mission_images_bgr[i] for i in indices]
# average_mission_image_bgr = averager(images)

In [ ]:
# plot_images([average_mission_image_bgr], ['average_mission_image_bgr'])

# BGR Matching Comparison

In [ ]:
plot_images(mission_images_bgr, ['mission_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
plot_images(histogram_mission_images_bgr, ['histogram_mission_images_bgr ' + str(mission_number) for mission_number in mission_numbers])

# HSV

In [ ]:
mission_images_hsv = []
histogram_mission_images_hsv = []

histogram_region_images_hsv = []

for i in range(0, len(mission_numbers)):
    hsv = cv2.cvtColor(mission_images_bgr[i].copy(), cv2.COLOR_BGR2HSV)
    mission_images_hsv.append(hsv)
    
    hsv = cv2.cvtColor(histogram_mission_images_bgr[i].copy(), cv2.COLOR_BGR2HSV)
    histogram_mission_images_hsv.append(hsv)
    
    hsv = cv2.cvtColor(histogram_region_images_bgr[i].copy(), cv2.COLOR_BGR2HSV)
    histogram_region_images_hsv.append(hsv)

# HSV Matching Comparison

In [ ]:
plot_images(mission_images_hsv, ['mission_images_hsv ' + str(mission_number) for mission_number in mission_numbers])
plot_images(histogram_mission_images_hsv, ['histogram_mission_images_hsv ' + str(mission_number) for mission_number in mission_numbers])
plot_images(histogram_region_images_hsv, ['histogram_region_images_hsv ' + str(mission_number) for mission_number in mission_numbers])

In [ ]:
# for i in range(0, len(mission_numbers)):
#     np.save(get_mission_file_path(mission_numbers[i]) + 'histogram_matched_image' + '.npy', histogram_mission_images[i])
#     np.save(get_mission_file_path(mission_numbers[i]) + 'histogram_matched_image_hsv' + '.npy', histogram_mission_images_hsv[i])

# bgr = np.load(get_mission_file_path(2) + 'histogram_matched_image' + '.npy')
# hsv = np.load(get_mission_file_path(2) + 'histogram_matched_image_hsv' + '.npy')
# plot_images([bgr, hsv], ['bgr', 'hsv'])

In [ ]:
# h = [mission_image_hsv[:, :, 0] for mission_image_hsv in histogram_mission_images_hsv]
# s = [mission_image_hsv[:, :, 1] for mission_image_hsv in histogram_mission_images_hsv]
# v = [mission_image_hsv[:, :, 2] for mission_image_hsv in histogram_mission_images_hsv]

In [ ]:
# plot_images(h, ['H ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(s, ['S ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(v, ['V ' + str(mission_number) for mission_number in mission_numbers])

# Shadow - Detection then Blackout

In [ ]:
shadow_masks = []
shadow_mission_images_hsv = []
smoothed_shadow_masks = []
smoothed_shadow_mission_images_hsv = []

for i in range(0, len(mission_numbers)):
    lower_shadow = np.array([69, 20, 1]) # from hsv_finder.py
    upper_shadow = np.array([129, 102, 34])
    mask = cv2.inRange(mission_images_hsv[i].copy(), lower_shadow, upper_shadow)
    shadow_masks.append(mask)
    image = cv2.bitwise_and(mission_images_hsv[i], mission_images_hsv[i], mask=mask).astype(np.uint8)
    shadow_mission_images_hsv.append(image)
     
    mask = copy.deepcopy(mask)
    mask = area_opening(mask, area_threshold=50)
    mask = cv2.dilate(mask, disk(radius=5), iterations=1)
    smoothed_shadow_masks.append(mask)
    smoothed_shadow_mission_images_hsv.append(cv2.bitwise_and(mission_images_hsv[i], mission_images_hsv[i], mask=mask).astype(np.uint8))

In [ ]:
# plot_images(shadow_masks, ['shadow_mask ' + str(mission_number) for mission_number in mission_numbers])
plot_images(shadow_mission_images_hsv, ['shadow_mission_images_hsv ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(smoothed_shadow_masks, ['smoothed_shadow_masks ' + str(mission_number) for mission_number in mission_numbers])
plot_images(smoothed_shadow_mission_images_hsv, ['smoothed_shadow_mission_images_hsv ' + str(mission_number) for mission_number in mission_numbers])

In [ ]:
# create all encompassing shadow mask

complete_shadow_mask = np.zeros((smoothed_shadow_masks[0].shape)).astype(int)
for i in range(0, len(shadow_masks)):
    complete_shadow_mask = cv2.bitwise_or(complete_shadow_mask, smoothed_shadow_masks[i].astype(int))

complete_shadow_mask = complete_shadow_mask.astype(np.uint8)
plot_images([complete_shadow_mask], ['complete_shadow_mask'], size_per_image=5)

In [ ]:
modified_mission_images_bgr = []
modified_mission_images_hsv = []

for i in range(0, len(mission_numbers)):
    modified_bgr = copy.deepcopy(histogram_mission_images_bgr[i])
    modified_hsv = copy.deepcopy(histogram_mission_images_hsv[i])

    try:
#         modified[smoothed_shadow_masks[i].astype(bool), :] = 0 # set to black
        modified_bgr[complete_shadow_mask.astype(bool), :] = 0 # set to black
        modified_hsv[complete_shadow_mask.astype(bool), :] = 0 # set to black
    except ValueError:
        print('not modified')
    
    modified_mission_images_bgr.append(modified_bgr)
    modified_mission_images_hsv.append(modified_hsv)

In [ ]:
# plot_images(mission_images_bgr, ['mission_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
plot_images(modified_mission_images_bgr, ['modified_mission_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(mission_images_hsv, ['mission_images_hsv ' + str(mission_number) for mission_number in mission_numbers])
plot_images(modified_mission_images_hsv, ['modified_mission_images_hsv ' + str(mission_number) for mission_number in mission_numbers])

# Ratio, Difference

In [ ]:
# 130, 123
box_w = 1
box_y = 123

x_lim = np.array([0, int((255-box_w)/2), 255 - int((255-box_w)/2), 255])
y_lim = np.array([0,  box_y, box_y, 255])
z = np.polyfit(x_lim, y_lim, 5) # R(0-255) --> R(0-255)
print(x_lim)
print(y_lim)
print(z)

nonlinear_poly = np.poly1d(z)

xp = np.linspace(0, 255, 256)
yp = nonlinear_poly(xp)

xp_adj = np.linspace(0, 255, 256)
yp_adj = nonlinear_poly(xp_adj)
mask = [num >= x_lim[1] and num <= x_lim[2] for num in xp_adj]
yp_adj[mask] = box_y

In [ ]:
plt.plot(x_lim, y_lim, 'k', xp, yp, 'r', xp, yp_adj, 'b')

In [ ]:
def detection_methods(img1, img2, method=''):
#     img1 = copy.deepcopy(cv2.cvtColor(img1, cv2.COLOR_HSV2GRAY))+1 # avoids / 0
#     img2 = copy.deepcopy(cv2.cvtColor(img2, cv2.COLOR_HSV2GRAY))+1
    img1 = copy.deepcopy(img1[:, :, :])+1
    img2 = copy.deepcopy(img2[:, :, :])+1
    threeD = True

    img1 = img1.astype(np.int16) # SUPER IMPORTANT - convert to int16 (or higher but necessary) to so that "result" array can preserve greater than +-255
    img2 = img2.astype(np.int16)
    
#     print('----')
    
    if method == 'difference':
        print('----------')
        if threeD:
            result = img1 - img2
            result += 255
            
            result = (result-np.min(result))*(255/(np.max(result) - np.min(result)))
            
            result = result.astype(np.uint8)
            result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
            
        else:
            
            result = img1 - img2
    #         print(np.min(result), np.max(result))

            result += 255 # max difference is 255 or -255, add 255 to bring everything positive
    #         print(np.min(result), np.max(result))

            #----------
            result = (result-np.min(result))*(255/(np.max(result) - np.min(result))) # linear 0-255 scaling

    #         hist_1, bins_1 = np.histogram(result.ravel(), 256, [0, 256])

            xp_adj = copy.deepcopy(result)
            yp_adj = nonlinear_poly(xp_adj) # nonlinear 0-255 scaling

    #         print(x_lim[1], x_lim[2])
            mask = cv2.inRange(xp_adj.copy(), np.array([x_lim[1]]).astype(np.uint8), np.array([x_lim[2]]).astype(np.uint8)).astype(bool)
            print(np.sum(mask)/np.prod(mask.shape)*100)

    #         result[mask] = box_y # for 2D

    #         yp_adj[mask] = box_y
    #         result = yp_adj
    #         print(result[0:2, 0:5])

    #         result = (result-np.min(result))*(255/(np.max(result) - np.min(result))) # linear 0-255 scaling because nonlinear scale limits may be slightly off
    #         print(np.min(result), np.max(result))

    #         hist_2, bins_2 = np.histogram(result.ravel(), 256, [0, 256])

            #----------
    
    elif method == 'ratio':
        result = img1 / img2
        print(np.min(result), np.max(result))
        
        result = (result-np.min(result))*(255/(np.max(result) - np.min(result)))
        print(np.min(result), np.max(result))

    else:
        print('NA')
        
    result = result.astype(np.uint8) # convert back to uint8 to be displayed
#     print(np.min(result), np.max(result))
    
    return result

In [ ]:
# using modified_mission_images_hsv[0]

adjacent_regions_change_hsv = []

for i in range(0, len(mission_numbers)):
    adjacent_regions_change_hsv.append(detection_methods(modified_mission_images_bgr[0], modified_mission_images_bgr[i], method='difference'))

In [ ]:
plot_images(adjacent_regions_change_hsv, ['adjacent_regions_change_hsv ' + str(mission_number) for mission_number in mission_numbers])

In [ ]:
masks = []
clean_masks_1 = []
clean_masks_final = []

for i in range(0, len(adjacent_regions_change_hsv)):
    mask = np.bitwise_not(cv2.inRange(adjacent_regions_change_hsv[i].copy(), np.array([box_y - 5]).astype(np.uint8), np.array([box_y + 5]).astype(np.uint8)).astype(bool))
    temp = np.zeros((mask.shape)).astype(np.uint8)
    temp[mask] = 255
    masks.append(temp)
    
    mask = cv2.erode(masks[i], disk(radius=2), iterations=1) # get rid of small stuff
    clean_masks_1.append(mask)
    mask = cv2.dilate(mask, disk(radius=5), iterations=1)
    clean_masks_final.append(mask)

In [ ]:
plot_images(masks, ['masks ' +  str(mission_number) for mission_number in mission_numbers])
# plot_images(clean_masks_1, ['clean_masks_1 ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(clean_masks_final, ['clean_masks_final ' + str(mission_number) for mission_number in mission_numbers])

In [ ]:
bbox_masks, num_bboxs, superimposed_images = create_bounding_boxes(clean_masks_final, histogram_mission_images_bgr, display_bbox=True, display_output=True, num_bbox_iterations=1, bbox_color=(255, 0, 0))

# Color Gradient

In [ ]:
color_buckets = create_color_buckets(colors=['#FF0000', '#00FF00'], num_buckets=10, display=True)
print(color_buckets)

# Roof - percent non-white roof pixels

In [ ]:
roof_masks = []
roof_region_images_hsv = []
smoothed_roof_masks = []
smoothed_roof_region_images_hsv = []

for i in range(0, len(mission_numbers)):
    lower_roof = np.array([0, 0, 0]) # from hsv_finder.py
    upper_roof = np.array([25, 45, 200])
    mask = cv2.inRange(histogram_region_images_hsv[i].copy(), lower_roof, upper_roof)
    roof_masks.append(mask)
    image = cv2.bitwise_and(histogram_region_images_hsv[i], histogram_region_images_hsv[i], mask=mask).astype(np.uint8)
    roof_region_images_hsv.append(image)
     
#     mask = copy.deepcopy(mask)
#     mask = area_opening(mask, area_threshold=50)
#     mask = cv2.dilate(mask, disk(radius=5), iterations=1)
#     smoothed_roof_masks.append(mask)
#     smoothed_roof_region_images_hsv.append(cv2.bitwise_and(histogram_region_images_hsv[i], histogram_region_images_hsv[i], mask=mask).astype(np.uint8))

In [ ]:
plot_images(roof_masks, ['roof_masks ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(histogram_region_images_bgr, ['histogram_region_images_bgr ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(smoothed_roof_masks, ['smoothed_roof_masks ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(smoothed_roof_region_images_hsv, ['smoothed_roof_region_images_hsv ' + str(mission_number) for mission_number in mission_numbers])

In [ ]:
percent_dark_pixels = []

for i in range(0, len(roof_masks)):
    percent_dark_pixels.append( np.sum(roof_masks[i]!=255) / np.prod(roof_masks[i].shape) * 100 )

plt.scatter(mission_numbers, percent_dark_pixels)
plt.xlabel('Mission Number')
plt.ylabel('% Dark Pixels')

# Texture

# PCA

In [ ]:
def svd(X):
    """
    Do SVD. You could use numpy SVD. 
    Your function should be able to handle black and white 
    images (N*D arrays) as well as color images (N*D*3 arrays)

    In the image compression, we assume that each coloum of the image is a feature. Image is the matrix X.
    
    Args:
        X: N * D array corresponding to an image (N*D*3 if color image)
    Return:
        U: N*N (*3 for color images)
        S: min(N, D)*1 (*3 for color images)
        V: D*D (*3 for color images)
    """
    
    D = X.shape[1]
    N = X.shape[0]

    if len(X.shape) == 2: #N x D
        U, S, V = np.linalg.svd(X)
        S = np.expand_dims(S, axis=1)
    elif len(X.shape) == 3: #N x D x 3, run through each dimension
        U = np.empty((N, N, 0))
        S = np.empty((np.min([N, D]), 0))
        V = np.empty((D, N, 0))
        
        for i in range(0, X.shape[2]):
            U_i, S_i, V_i = np.linalg.svd(X[:, :, i])            
            U = np.append(U, np.expand_dims(U_i, axis=2), axis=2)
            S = np.append(S, np.expand_dims(S_i, axis=1), axis=1)
            V = np.append(V, np.expand_dims(V_i, axis=2), axis=2)

    return U, S, V

def rebuildsvd(U, S, V, k):
    """
    Rebuild SVD by k componments.
    Args: 
        U: N*N (*3 for color images) 
        S: min(N, D)*1 (*3 for color images)
        V: D*D (*3 for color images)  
        k: int corresponding to number of components
    Return:
        Xrebuild: N*D array of reconstructed image (N*D*3 if color image)
        
    Hint: numpy.matmul may be helpful for reconstructing color images
    """
    
    D = V.shape[0]
    N = U.shape[0]

    if len(U.shape) == 2: #N x D
        Xrebuild = np.matrix(U[:, 0:k]) * np.diagflat(S[0:k]) * np.matrix(V[0:k, :])
    elif len(U.shape) == 3:
        Xrebuild = np.empty((N, D, 0))
        for i in range(0, U.shape[2]):
            Xrebuild_i = np.matrix(U[:, 0:k, i]) * np.diagflat(S[0:k, i]) * np.matrix(V[0:k, :, i])           
            Xrebuild = np.append(Xrebuild, np.expand_dims(Xrebuild_i, axis=2), axis=2)
    
    return Xrebuild

def compression_ratio(X, k):
    """
    Compute compression of an image: (num stored values in compressed)/(num stored values in original)
    Args: 
        X: N * D array corresponding to an image (N * D * 3 if color image)
        k: int corresponding to number of components
    Return:
        compression_ratio: float of proportion of storage used by compressed image
    """

    #http://timbaumann.info/svd-image-compression-demo/
    H = X.shape[0]
    W = X.shape[1]
    
    return (W*k + k + k*H) / (H*W)

def recovered_variance_proportion(S, k):
    """
    Compute the proportion of the variance in the original matrix recovered by a rank-k approximation
    
    Args:
       S: min(N, D)*1 (*3 for color images) of singular values for the image
       k: int, rank of approximation
    Return:
       recovered_var: int (array of 3 ints for color image) corresponding to proportion of recovered variance
    """

    #variance recovered = variance of top K components / variance of all components
    
    variance_recovered = np.power(S, 2)
    
    if S.shape[1] == 1:
        return np.sum(variance_recovered[0:k], axis=0)[0] / np.sum(variance_recovered, axis=0)[0]
    elif S.shape[1] == 3:
        return np.sum(variance_recovered[0:k, :], axis=0) / np.sum(variance_recovered, axis=0)

In [ ]:
# from sklearn.decomposition import PCA

pca_images = []
for i in range(0, len(mission_numbers)):
    U, S, V = svd(mission_images[i][0:470, 0:470, :]) # needs to be square
    component_num = [1, 5, 10, 50, 100, 500]

    fig = plt.figure(figsize=(30, 30))

    # plot several images
    pca_list = []
    j = 0
    for k in component_num:
        img_rebuild = rebuildsvd(U, S, V, k)/255
#         c = np.around(compression_ratio(image, k), 4)
#         r = np.around(recovered_variance_proportion(S, k), 3)
        ax = fig.add_subplot(1, 6, j+1, xticks=[], yticks=[])
        ax.imshow(img_rebuild)
        ax.set_title(f"Mission {mission_numbers[i]} - {k} Components")
#         ax.set_xlabel(f"Compression: {np.around(c,4)},\nRecovered Variance: R: {r[0]} G: {r[1]} B: {r[2]}")
        j += 1
        pca_list.append((img_rebuild*255).astype(np.uint8))
    
    pca_images.append(pca_list)

In [ ]:
hsv_pca_images = []
for i in range(0, len(pca_images)):
    hsv_pca_list = []
    for j in range(0, len(pca_images[i])):
        hsv_pca_list.append(cv2.cvtColor(pca_images[i][j], cv2.COLOR_BGR2HSV))
    hsv_pca_images.append(hsv_pca_list)

In [ ]:
# for i in range(0, len(hsv_pca_images)):
#     plot_images(hsv_pca_images[i])

In [ ]:
temp = []
for i in range(0, len(mission_numbers)-1):
    temp.append(cv2.cvtColor(np.absolute(pca_images[i][0] - pca_images[i+1][0]), cv2.COLOR_BGR2GRAY))

plot_images(temp)

In [ ]:
temp = []
for i in range(0, len(mission_numbers)-1):
    temp.append(np.absolute(hsv_pca_images[i][3] - hsv_pca_images[i+1][3]))

plot_images(temp)

# Contours

In [ ]:
import imutils
from skimage.measure import compare_ssim

In [ ]:
imageA = region_images[0]
imageB = region_images[1]
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

(score, diff) = compare_ssim(grayA, grayB, full=True)
diff = (diff*255).astype("uint8")
print("SSIM: {}".format(score)) # [-1, 1] 1 perfect match

In [ ]:
thresh = cv2.threshold(diff, 100, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

In [ ]:
for c in cnts:
    # compute the bounding box of the contour and then draw the
    # bounding box on both input images to represent where the two
    # images differ
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(imageA, (x, y), (x+w, y+h), (0, 0, 255), 1)
    cv2.rectangle(imageB, (x, y), (x+w, y+h), (0, 0, 255), 1)

plot_images([imageA, imageB, grayA, grayB, diff, thresh], ['imageA', 'imageB', 'grayA', 'grayB', 'diff', 'thresh'])

In [ ]:
thresholds = []

for i in range(0, len(adjacent_regions_change_hsv)):
    thresh = cv2.threshold(adjacent_regions_change_hsv[i], 100, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    thresholds.append(thresh)

In [ ]:
plot_images(thresholds)